# <span style="color:teal">RBFE Network Tutorial - Setup</span>
This is the RBFE (Relative Binding Free Energy) setup workflow jupyter notebook for the September 2022 CCPBioSim Workshop.
It includes core as well as <span style="color:purple">extra</span> options.

**<span style="color:teal">Authors</span>**
 - Anna Herz (@annamherz)
 - This is adapted from the FEP BioSimSpace Tutorial written by Jenke Scheen (https://github.com/michellab/BioSimSpaceTutorials/tree/main/04_fep).

**<span style="color:teal">Reading Time:</span>**
~ XX

##### <span style="color:teal">Required knowledge</span> 
 - Basic python
 - Part 1 of this workshop (Introduction to Alchemistry with BioSimSpace)
    - this should include basic knowledge of the principles behind RBFE

##### <span style="color:teal">Learning objectives</span>  
 - Setup an FEP (Free Energy Perturbation) pipeline using BioSimSpace and SOMD.
 - Analyse and plot the results.

### <span style="color:teal">Table of Contents</span>  
1. [Introduction](#intro)       
    1.1 [The system: TYK2](#tyk2)     
2. [Setup](#setup)   
    2.1 [Choosing the parameters](#parameters)     
    2.2 [Setting up the network](#perts)    
    2.3 [Preparing for the pipeline](#loading)     
    2.4 [Parallelisation](#loading)     

<span style="color:pink">Further reading </span> references some sections of the [LiveComs Best Practices for Alchemical Free Energy Calculations](https://livecomsjournal.org/index.php/livecoms/article/view/v2i1e18378).

**<span style="color:teal">Jupyter Cheat Sheet</span>**
- To run the currently highlighted cell and move focus to the next cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- To run the currently highlighted cell and keep focus in the same cell, hold <kbd>&#x21E7; Ctri</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- To get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;
- You can find the full documentation at [biosimspace.org](https://biosimspace.org).


Below, we will import the neccessary libraries and also define all the folder locations for our run. This ensures that all of our files are written to where we want them to be.

In [1]:
# import libraries
import BioSimSpace as BSS
import os
import glob
import csv
import numpy as np

# define all the folder locations
main_folder =  os.getcwd()
print(main_folder)
# scripts should be located in:
scripts_folder = f"{main_folder}/scripts"
# other folders
input_dir = f"{main_folder}/inputs"
path_to_ligands = f"{main_folder}/inputs/ligands"


Sending anonymous Sire usage statistics to http://siremol.org.
For more information, see http://siremol.org/analytics
To disable, set the environment variable 'SIRE_DONT_PHONEHOME' to 1
To see the information sent, set the environment variable 
SIRE_VERBOSE_PHONEHOME equal to 1. To silence this message, set
the environment variable SIRE_SILENT_PHONEHOME to 1.



/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:500: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/anna/anaconda3/envs/biosimspace-dev/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


/home/anna/Documents/september_2022_workshops/bssccpbiosim2022/RBFE_tutorial


### <span style="color:teal">1. Introduction</span>
<a id="intro"></a>

Computational chemists can support structure-activity relationship studies in medicinal chemistry by making computer models that can predict binding affinity of ligands to proteins. One of the most popular techniques for this is Free Energy Perturbation (FEP), which relies on simulation alchemical transformations of ligands in a congeneric series, simulating them both in a protein target and in just a waterbox. Relative free energies of binding (ΔΔG in kcal/mol) can then be computed by simply subtracting the ΔG (in protein) and the ΔG (in water). The basics of setting up and running an FEP calculation with BSS have already been covered in the `introduction_to_alchemistry` part of this workshop.

In the previous section of the workshop, we only looked at a singular perturbation. However, typically in FEP the goal is to predict free energies of binding for a collection of ligands (normally 10-20). Although methods exist (such as absolute FEP) that can predict these energies directly (i.e. ΔGbind), these are often complicated and computationally expensive, whilst relative binding free energies are more accessible to run and give useful results as we are usually considering a congeneric series of ligands. Because we calculate relative energies, we therefore have to transform the between pairs of ligands in both the fee and bound phase (hence the name free energy perturbation. Typically, smaller (i.e. fewer heavy atoms) transformations are more reliable which means that for a ligand series we want to selectively make combinations of ligands to cover the whole series. In FEP, we do this using perturbation networks, typically generated by FEP softwares. Although generate these networks can be done by hand, it is typically better to do it programmatically to save time and create better networks (transformation reliability does not depend just on transformation size, but also a series of other unfavourable moiety transformations).   

In this part of the workshop we will focus on how to setup this network for a series of ligands and also write all files necessary to further prepare our FEP simulations. The overall pipeline used is shown in the figure below.   

![pipeline](images/fep_pipeline.png)

Because preparing ligands and proteins for FEP can already require some heavy computation, this will be the first process that will run on a cluster. Then, after running and processing the FEP outputs, we can download the results back to our local workstation. There, the analysis notebook uses FreeEnergyAnalysis to process FEP predictions and generate plots. This is also explained in more detail at the end of the notebook.


#### <span style="color:teal">1.1. The system: TYK2</span>
<a id="tyk2"></a>

For this tutorial we will be using TYK2, a common benchmarking set in the FEP field, first used by Schrödinger in their [2015 FEP+ paper](https://pubs.acs.org/doi/full/10.1021/ja512751q).


![tyk2_49](images/tyk2_protlig.png)

We can also view this protein below:


In [ ]:
BSS.Notebook.View("input/protein.pdb").system()

#### <span style="color:teal">2. Setting up the Network</span>
<a id="setup"></a>

##### <span style="color:teal">2.1 Choosing the parameters for the FEP runs</span>
<a id="parameters"></a>

Running the below cell will generate the input nodes that can be used to choose various parameters for the FEP network.

This includes the forcefields used (ligand, protein, water), the number of lambda windows, and the threshold for the LOMAP score. This threshold will be used to define a perturbation as being either 'regular' or 'difficult', and consequently how many lambda windows will be used. Default parameters are selected for all nodes.


In [ ]:
node = BSS.Gateway.Node("A node to create input files for molecular dynamics simulation.")

node.addInput("Ligand FF", BSS.Gateway.String(help="Force field to parameterise ligands with.",
                                             allowed=["GAFF1", "GAFF2"],
                                             default="GAFF2"))

node.addInput("Protein FF", BSS.Gateway.String(help="Force field to parameterise the protein with.",
                                             allowed=["FF03", "FF14SB", "FF99", "FF99SB", "FF99SBILDN"],
                                             default="FF14SB"))

node.addInput("Water Model", BSS.Gateway.String(help="Water model to use.",
                                             allowed=["SPC", "SPCE", "TIP3P", "TIP4P", "TIP5P"],
                                             default="TIP3P"))

node.addInput("Box Edges", BSS.Gateway.String(help="Size of water box around molecular system.",
                                             allowed=["20*angstrom", "25*angstrom", "30*angstrom", "35*angstrom", "45*angstrom", "5*nm", "7*nm", "10*nm"],
                                             default="20*angstrom"))

node.addInput("Box Shape", BSS.Gateway.String(help="Geometric shape of water box.",
                                             allowed=["cubic", "truncatedOctahedron"],
                                             default="cubic"))

node.addInput("Run Time", BSS.Gateway.String(help="The sampling time per lambda window.",
                                             allowed=["10*ps", "100*ps", "1*ns", "2*ns", "3*ns", "4*ns", "5*ns", "8*ns", "10*ns", "12*ns", "15*ns"],
                                             default="2*ns"))

node.addInput("FEP Engine", BSS.Gateway.String(help="Engine to run FEP with.",
                                             allowed=[e.upper() for e in BSS.FreeEnergy.engines()],
                                             default="SOMD"))

node.addInput("LambdaWindows", BSS.Gateway.String(help="The number of lambda windows for regular transformations.",
                                             allowed=["3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"],
                                             default="11"))

node.addInput("DiffLambdaWindows", BSS.Gateway.String(help="The number of lambda windows for difficult transformations.",
                                             allowed=["4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20"],
                                             default="17"))
                                             
node.addInput("LOMAP Threshold", BSS.Gateway.String(help="The LOMAP score threshold to define difficult transformations.",
                                             allowed=["0.1", "0.2", "0.3", "0.4", "0.5", "0.6", "0.7", "0.8", "0.9"],
                                             default="0.4"))

node.showControls()

After the parameters are chosen, the next step for generating an FEP network is to check the structure of the protein and ligands for which this will be run. 


The ligand files are provided in the inputs/ligands folder. They were generated using [FEGrow](https://github.com/cole-group/FEgrow).


##### <span style="color:teal">1.2. The FEP Network</span>  
<a id="perts"></a>

Because we calculate relative energies, we therefore have to transform the between pairs of ligands in both the fee and bound phase (hence the name free energy perturbation. Typically, smaller (i.e. fewer heavy atoms) transformations are more reliable which means that for a ligand series we want to selectively make combinations of ligands to cover the whole series. In FEP, we do this using perturbation networks, typically generated by FEP softwares. Although generate these networks can be done by hand, it is typically better to do it programmatically to save time and create better networks (transformation reliability does not depend just on transformation size, but also a series of other unfavourable moiety transformations).   


The reliability of FEP calculations is typically higher for transformations with fewer heavy atom changes and some other rules (e.g. no ring formations). LOMAP is a package that contains heuristics for this and is able to come up with a network for us; BioSimSpace contains a function generateNetwork() that uses LOMAP to come up with the perturbation network that visualises the desired perturbations. Additionally, a LOMAP score is calculated which quantifies the likeliness that the FEP prediction will be reliable (0 means unlikely, 1 means highly likely).   

 <span style="color:pink">Further reading </span>: 7.1 (perturbation maps, Figure 5)

In [ ]:
#generate transformation network based on ligands
path_to_ligands ="inputs/ligands"
ligand_files = sorted(glob.glob(f"{path_to_ligands}/*.sdf"))

ligands = []
ligand_names = []

for filepath in ligand_files:
    # append the molecule object to a list.
    ligands.append(BSS.IO.readMolecules(filepath)[0])
    
    # append the molecule name to another list so that we can use the name of each molecule in our workflow.
    ligand_names.append(filepath.split("/")[-1].replace(".sdf",""))

transformations, lomap_scores = BSS.Align.generateNetwork(ligands, plot_network=True, names=ligand_names)


BioSimSpace uses NetworkX which works with node indices for node names. Let's adjust our list of edges to have ligand names instead of ligand indices. We also generate a dictionary that contains all of the network information we need downstream.

In [ ]:
pert_network_dict = {}
transformations_named = [(ligand_names[transf[0]], ligand_names[transf[1]]) for transf in transformations]
for score, transf in sorted(zip(lomap_scores, transformations_named)):
    pert_network_dict[transf] = score
    print(transf, score)


Unfortunately, even cutting-edge perturbation network generators such as LOMAP require some manual tweaking. In some cases, a ligand will have poor LOMAP scores on its edges and is therefore likely to be unreliable. To increase its reliability we often want to create an additional edge to this ligand, which can be easily done by appending the edge to our list of edges. Additionally, we usually want to avoid any ring breaking/forming transformations.

To this end, we can introduce a ligand 'intermediate_H' that contains no atoms on the R-group. This makes it easier for the FEP code to perturb to cyclical R-groups. Using this type of network manipulation allows the user to include some cycles into the network, which is good for statistical performance of FEP predictions.

Copy this ligand from the inputs/ligands/intermediate folder into the inputs/ligands folder and rerun the above network generation.

Below, add any other perturbations, either with the intermediate, or any other ligands to complete a cycle.


In [ ]:
pert_network_dict[('ejm47', 'ejm49')] = 0.1

pert_network_dict

Another way to manipulate the dictionary is to remove entries (i.e. remove edges from the network). When we look at the edges suggested by LOMAP that have low scores, we see that some are likely to be unreliable in FEP. Luckily, this is because cycle formation/removal in a single-topology-style FEP must be from/to a hydrogen, not a carbon atom. This is why we have introduced 'intermediate_H', so this has largely been solved by generating the network again above. Still, the below syntax can be used if any edges are to be removed.

In [ ]:
for key in [('A', 'B'), ('C', 'D')]:
    del pert_network_dict[key]
pert_network_dict

##### <span style="color:teal">1.3. Preparing for the FEP pipeline</span>  

Once we have our protein, ligands, and have planned the network, we need to write all the files we need for running the FEP Pipeline.

This includes the following files:
 - ligands.dat , which includes all the ligands that must be prepared
 - network.dat , which includes all the perturbations and the number of lambda windows
 - protocol.dat , which includes the details of the protocol used

In [ ]:
# write ligands file.
with open("ligands.dat", "w") as ligands_file:
    writer = csv.writer(ligands_file)
    for lig in ligand_names:
        writer.writerow([lig])

# write perts file. Base the lambda schedule on the file generated in the previous cell.
np.set_printoptions(formatter={'float': '{: .4f}'.format})

# from protocol, derive the engine we want to use on the cluster.
engine = node.getInput('FEP Engine').upper()

with open("network.dat", "w") as network_file:

    writer = csv.writer(network_file, delimiter=" ")
    
    for pert, lomap_score in pert_network_dict.items():
        # based on the provided (at top of notebook) lambda allocations and LOMAP threshold, decide allocation.
        if lomap_score == None or lomap_score < float(node.getInput("LOMAP Threshold")):
            num_lambda = node.getInput("DiffLambdaWindows")
        else:
            num_lambda = node.getInput("LambdaWindows")
            
       
        # given the number of allocated lambda windows, generate an array for parsing downstream.
        lam_array_np = np.around(np.linspace(0, 1, int(num_lambda)), decimals=5)

        # make the array into a format readable by bash.
        lam_array = str(lam_array_np).replace("[ ", "").replace("]", "").replace("  ", ",").replace('\n', '')

        # write out both directions for this perturbation.
        writer.writerow([pert[0], pert[1], len(lam_array_np), lam_array, engine])
        writer.writerow([pert[1], pert[0], len(lam_array_np), lam_array, engine])         

# create protocol. 
protocol = [
    f"ligand forcefield = {node.getInput('Ligand FF')}",
    f"protein forcefield = {node.getInput('Protein FF')}",
    f"solvent = {node.getInput('Water Model')}",
    f"box edges = {node.getInput('Box Edges')}",
    f"box type = {node.getInput('Box Shape')}",
    f"protocol = default",
    f"sampling = {node.getInput('Run Time')}",
    f"engine = {node.getInput('FEP Engine').upper()}"
]

# write protocol to file.
with open("protocol.dat", "w") as protocol_file:
    writer = csv.writer(protocol_file)

    for prot_line in protocol:
        
        writer.writerow([prot_line])

We can print these below to check them.

In [2]:
print("Ligands...")
f = open(f"ligands.dat")
for row in csv.reader(f):
    print(str(row))

print('\n')
print("Network...")
f = open(f"network.dat")
for row in csv.reader(f):
    print(str(row))

print('\n')
print("Protocol...")
f=open(f"protocol.dat")
for row in csv.reader(f):
    print(str(row))

Ligands...
['ejm31']
['ejm42']
['ejm43']
['ejm44']
['ejm45']
['ejm46']
['ejm47']
['ejm48']
['ejm49']
['ejm50']
['ejm51']
['ejm52']
['ejm53']
['ejm54']
['ejm55']
['intermediate_H']


Network...
['ejm54 intermediate_H 11 0.0000', '0.1000', '0.2000', '0.3000', '0.4000', '0.5000', '0.6000', '0.7000', '0.8000', '0.9000', '1.0000 SOMD']
['intermediate_H ejm54 11 0.0000', '0.1000', '0.2000', '0.3000', '0.4000', '0.5000', '0.6000', '0.7000', '0.8000', '0.9000', '1.0000 SOMD']
['ejm45 intermediate_H 11 0.0000', '0.1000', '0.2000', '0.3000', '0.4000', '0.5000', '0.6000', '0.7000', '0.8000', '0.9000', '1.0000 SOMD']
['intermediate_H ejm45 11 0.0000', '0.1000', '0.2000', '0.3000', '0.4000', '0.5000', '0.6000', '0.7000', '0.8000', '0.9000', '1.0000 SOMD']
['ejm53 intermediate_H 11 0.0000', '0.1000', '0.2000', '0.3000', '0.4000', '0.5000', '0.6000', '0.7000', '0.8000', '0.9000', '1.0000 SOMD']
['intermediate_H ejm53 11 0.0000', '0.1000', '0.2000', '0.3000', '0.4000', '0.5000', '0.6000', '0.7000', '0

##### <span style="color:teal">Parallelisation - Running the MD simulations</span>

Once all the files are written, the folder can be copied to a computing cluster so that the simulations can be parallelised. A simple example script for this for a slurm cluster (without any analysis) is included in the scripts directory.

This calls the following scripts in order:

 - Ligand preparation (BSSligprep.py) - The ligand and protein are paramaterised, combined, and solvated. Equilibration is carried out.

 - FEP preparation (BSSprepFEP.py) - For the perturbation, the ligands are mapped according to their maximum common substructure, and a perturbable system is created. The folders for the FEP run for SOMD are written.

 - Running the production windows (runFEP.sh) - As each lambda window can be run independantly of other lambda windows, this is where most of the parallelisation takes place. Each window is submitted as part of a slurm array job.


More detailed scripts, including one for an LSF cluster, are in the [BioSimSpace Tutorials](https://github.com/michellab/BioSimSpaceTutorials/tree/main/04_fep/execution_model/scripts).

As these would take too long to run in the span of a workshop, outputs from the runs above are in the outputs directory. We will analyse these in the next part of the tutorial.